In [9]:
import os
import platform
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import datetime as dt
import pathlib
import pickle #to save files
from itertools import product
from scipy.stats import skew, kurtosis, wilcoxon
from scipy.signal import butter, welch, filtfilt, resample
from scipy import stats
# import xgboost as xgb
# from xgboost.sklearn import XGBClassifier #this is the SKlearn wrapper
from sklearn.metrics import confusion_matrix, auc, f1_score, roc_auc_score, precision_score, recall_score, precision_recall_curve
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn import svm
import time

from PreprocessFcns import *

from mpl_toolkits.mplot3d import Axes3D
%matplotlib inline

sns.set_context('talk',font_scale=1)
#-- For interactive plots--
# from mpl_toolkits.mplot3d import Axes3D
# %matplotlib notebook

In [11]:
from sklearn.model_selection import LeaveOneGroupOut
from sklearn import preprocessing
from sklearn import neighbors, linear_model
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, ExtraTreesClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import KFold, StratifiedKFold, cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn import metrics
from sklearn.linear_model import ElasticNetCV, LogisticRegression
from sklearn.metrics import roc_curve, auc, roc_auc_score
from sklearn.metrics import confusion_matrix

## Helper Functions

In [6]:
def LOSOCV(Data,X,y,groups,models,LOin=0):

    subj = LeaveOneGroupOut() 
    results = pd.DataFrame(data=None,columns=['model','f1','auprc','auroc'])
    groupres = {} #results on each group for each model


    #train multiple classifiers
    for m in models:
        acc_all=[]; acc_train=[] 
        f1_test_all=[]; prec_all=[]; rec_all=[]; spec_all=[]; auprc_all=[]; auroc_train_all=[]; 
        auroc_all=[]; fpr_all=[]; tpr_all=[]; fi_all=[]

        clf = m[0]; model_name = m[1]            
        print('Training %s'%model_name)
        s = 0
        #LOSO CV for current classifier
        for train_index, test_index in subj.split(X, y, groups):
        #leave one in:
            if LOin:
                tridx = train_index.copy()
                train_index = test_index.copy()
                test_index = tridx
            Xtr, Xte = X[train_index], X[test_index]
            ytr, yte = y[train_index], y[test_index]
            if len(np.unique(ytr))<2: #skip if no positive or negative examples are available for training
                print('only 1 class available in train data - skipping')
                continue
            clf.fit(Xtr,ytr)
            ypred = clf.predict(Xte)
            yscore = clf.predict_proba(Xte)
            yscore = yscore[:,1]
            
            #accuracy on train set
            ypred_train = clf.predict(Xtr)
            acc_train.append(sum(ypred_train==ytr)/len(ytr))
            auroc_train = roc_auc_score(ytr,clf.predict_proba(Xtr)[:,1])
            
            #f1-score, prec, recall, specificity, auprc, auroc
            f1_test_all.append(f1_score(yte,ypred))
            precision, recall, _ = precision_recall_curve(yte,yscore)
            auprc = auc(recall,precision)
            if len(np.unique(yte))>1:
                auroc = roc_auc_score(yte,yscore)
            else:
                print('only 1 class in test data - cannot compute roc curve')
                auroc = np.nan
            spec = sum((ypred==0) & (yte==0))/sum(yte==0)


            prec_all.append(precision_score(yte,ypred))
            rec_all.append(recall_score(yte,ypred))
            auprc_all.append(auprc)
            auroc_all.append(auroc)
            auroc_train_all.append(auroc_train)
            spec_all.append(spec)
            
            #the recall per task - TO BE COMPLETED
            
#             tasks=Data.iloc[test_index,:].Task
#             for t in tasks.unique():
#                 tasks
            
            #compute ROC points at fixed fpr (to plot error bars)
            fpr=np.linspace(0,1,101); tpr=[]

            if len(np.unique(yte))>1:                
                nscores = np.sort(np.column_stack((yscore[yte==0],yte[yte==0])),axis=0)
                neg_counts = sum(yte==0)
                for f in fpr:
                    ind = neg_counts-int(neg_counts*f)-1
                    t = (nscores[ind])[0]
                    if f==1:
                        t = 0
                    tpr_t = sum(yscore[yte==1]>t) / sum(yte==1)
                    tpr.append(tpr_t) 

            fpr = np.asarray(fpr); tpr = np.asarray(tpr)
            fpr_all.append(fpr); tpr_all.append(tpr)
            
            #store feature importance
            if model_name != 'SVM':
                fi_all.append(clf.feature_importances_)
            
            print('\nSubj/Visit %d, prec=%.3f, rec=%.3f, Spec=%.3f, auroc_train=%.3f, auroc=%.3f'%(s,precision_score(yte,ypred),recall_score(yte,ypred),
                                                                                 spec,auroc_train,auroc))
            s+=1

        print('f1_test=%.3f+/-%.3f, prec=%.3f+/-%.3f, rec=%.3f+/-%.3f, auprc=%.3f+/-%.3f, auroc=%.3f+/-%.3f'%(
        np.nanmean(f1_test_all),np.nanstd(f1_test_all),
        np.nanmean(prec_all),np.nanstd(prec_all), np.nanmean(rec_all),np.nanstd(rec_all),
        np.nanmean(auprc_all),np.nanstd(auprc_all), np.nanmean(auroc_all),np.nanstd(auroc_all)))
        
        #group results for each model
        groupres[model_name] = {'f1':f1_test_all, 'auprc':auprc_all, 'auroc':auroc_all, 'tpr':tpr_all, 'fpr':fpr_all, 
                                'rec':rec_all, 'spec':spec_all, 'fi':fi_all}
        
        #mean across groups for each model
        r = pd.DataFrame({'model':model_name, 'f1':np.nanmean(f1_test_all), 'auprc':np.nanmean(auprc_all), 'auroc':np.nanmean(auroc_all)}
                        ,index=[0])
        results = pd.concat((results,r))
        
    return results,groupres 
    
def plot_roc(tpr_all,fpr,roc_auc,ax=None,plotname=None,col=None):
    #plot mean ROC across subjects (need to add shaded conf interval)
    tprmu = np.mean(np.asarray(tpr_all),axis=0)
    tpr=np.asarray(tpr_all)
    fpr=np.reshape(fpr,(1,-1))
    tprmu=np.reshape(tprmu,(1,-1))
    label=pd.Series(data = ['%s - AUC = %0.3f' % (plotname,roc_auc)]*len(fpr))
    if plotname=='Threshold':
        ls = '-'
    else:
        ls='-'
    if ax == None:
        ax = sns.tsplot(data=tpr,time=fpr,ci=95,condition=label,legend=True,color=col,lw=3,linestyle=ls)
    else:
        sns.tsplot(data=tpr,time=fpr,ci=95,condition=label, legend=True,ax=ax,color=col,lw=3,linestyle=ls)
             
    lw = 3
    
    ax.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
    ax.set_xlim([-0.05, 1.0])
    ax.set_ylim([0.0, 1.05])
    ax.set_xlabel('False Positive Rate',fontsize=16)
    ax.set_ylabel('True Positive Rate',fontsize=16)
    ax.legend(loc='lower right')
    
    return ax

In [20]:
# get all data and remove useless rows
Xtrain_t = pd.read_pickle('//FS2.smpp.local/RTO/CIS-PD Study/MJFF Curation/ClinicSmartWatchFeatures_HP.pkl')
Xtrain_b = pd.read_pickle('//FS2.smpp.local/RTO/CIS-PD Study/MJFF Curation/ClinicSmartWatchFeatures_HP+LP.pkl')
Xtrain_t = Xtrain_t[Xtrain_t.Subject!=1020]
Xtrain_b = Xtrain_b[Xtrain_b.Subject!=1020]
Xtrain_t = Xtrain_t[~Xtrain_t['Tremor'].isnull()]
Xtrain_b = Xtrain_b[~Xtrain_b['Bradykinesia'].isnull()]
Xtrain_t = Xtrain_t.reset_index(drop = True)
Xtrain_b = Xtrain_b.reset_index(drop = True)

clf_tremor = RandomForestClassifier(n_estimators=100, max_features = 0.5, random_state=2)

clf_brady = RandomForestClassifier(n_estimators=100, max_features = 0.5, random_state=2)

In [17]:
indp_t = Xtrain_t['Tremor']
indp_t = indp_t.values
ytrain_tremor = indp_t.astype(int)
ytrain_tremor[ytrain_tremor>0] = 1

LOSOCV(Xtrain_t, Xtrain_t.iloc[:,5:].values, ytrain_tremor, Xtrain_t.Subject.values,[(clf_tremor, 'Random Forest')])

Training Random Forest

Subj/Visit 0, prec=0.466, rec=0.358, Spec=0.814, auroc_train=1.000, auroc=0.632

Subj/Visit 1, prec=0.296, rec=0.526, Spec=0.876, auroc_train=1.000, auroc=0.744

Subj/Visit 2, prec=0.891, rec=0.132, Spec=0.972, auroc_train=1.000, auroc=0.706

Subj/Visit 3, prec=0.068, rec=0.267, Spec=0.911, auroc_train=1.000, auroc=0.668

Subj/Visit 4, prec=0.231, rec=0.040, Spec=0.953, auroc_train=1.000, auroc=0.335

Subj/Visit 5, prec=0.677, rec=0.137, Spec=0.962, auroc_train=1.000, auroc=0.718

Subj/Visit 6, prec=0.591, rec=0.275, Spec=0.963, auroc_train=1.000, auroc=0.825

Subj/Visit 7, prec=0.213, rec=0.222, Spec=0.946, auroc_train=1.000, auroc=0.719

Subj/Visit 8, prec=0.337, rec=0.248, Spec=0.927, auroc_train=1.000, auroc=0.688

Subj/Visit 9, prec=0.631, rec=0.283, Spec=0.915, auroc_train=1.000, auroc=0.746

Subj/Visit 10, prec=0.616, rec=0.361, Spec=0.818, auroc_train=1.000, auroc=0.727

Subj/Visit 11, prec=0.833, rec=0.333, Spec=0.993, auroc_train=1.000, auroc=0.952

Su

C:\Users\nshawen\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true samples.
  'recall', 'true', average, warn_for)
C:\Users\nshawen\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\ranking.py:444: RuntimeWarning: invalid value encountered in true_divide
  recall = tps / tps[-1]
C:\Users\nshawen\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\ranking.py:94: RuntimeWarning: invalid value encountered in less
  if np.any(dx < 0):
C:\Users\nshawen\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1137: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples.
  'recall', 'true', average, warn_for)
C:\Users\nshawen\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1137: UndefinedMetricWarning: Recall is ill-defined and being se

only 1 class in test data - cannot compute roc curve

Subj/Visit 16, prec=0.000, rec=0.000, Spec=0.917, auroc_train=1.000, auroc=nan

Subj/Visit 17, prec=0.349, rec=0.101, Spec=0.865, auroc_train=1.000, auroc=0.328

Subj/Visit 18, prec=0.826, rec=0.396, Spec=0.856, auroc_train=1.000, auroc=0.652

Subj/Visit 19, prec=0.940, rec=0.548, Spec=0.880, auroc_train=1.000, auroc=0.862

Subj/Visit 20, prec=0.855, rec=0.266, Spec=0.927, auroc_train=1.000, auroc=0.681

Subj/Visit 21, prec=0.027, rec=0.286, Spec=0.934, auroc_train=1.000, auroc=0.792
f1_test=0.304+/-0.196, prec=0.469+/-0.320, rec=0.274+/-0.170, auprc=0.458+/-0.285, auroc=0.682+/-0.184


(           model      f1     auprc     auroc
 0  Random Forest  0.3045  0.457656  0.682329,
 {'Random Forest': {'f1': [0.40471512770137524,
    0.3787878787878788,
    0.2300469483568075,
    0.10810810810810811,
    0.06818181818181819,
    0.22727272727272724,
    0.3754512635379061,
    0.21768707482993194,
    0.2857142857142857,
    0.3905325443786983,
    0.45503791982665226,
    0.47619047619047616,
    0.4766839378238341,
    0.6301369863013699,
    0.0,
    0.13438735177865613,
    0.0,
    0.15652173913043477,
    0.5353982300884956,
    0.692532942898975,
    0.40561224489795916,
    0.05],
   'auprc': [0.44996359295603866,
    0.24516467869462297,
    0.7951607415176183,
    0.04617210615149851,
    0.21245818689673682,
    0.5609258095389581,
    0.4391251785423096,
    0.1764515250944585,
    0.3168349874968212,
    0.5670410742655271,
    0.6310720451646094,
    0.6765686565686565,
    0.5642988257082963,
    0.893485481202369,
    0.003128372054203256,
    0.1834908537

In [21]:
indp_b = Xtrain_b['Bradykinesia']
indp_b = indp_b.values
ytrain_brady = indp_b.astype(int)
ytrain_brady[ytrain_brady>0] = 1

LOSOCV(Xtrain_b, Xtrain_b.iloc[:,5:].values, ytrain_brady, Xtrain_b.Subject.values,[(clf_brady, 'Random Forest')])

Training Random Forest

Subj/Visit 0, prec=0.523, rec=0.749, Spec=0.526, auroc_train=1.000, auroc=0.690

Subj/Visit 1, prec=0.416, rec=0.578, Spec=0.643, auroc_train=1.000, auroc=0.662
only 1 class in test data - cannot compute roc curve

Subj/Visit 2, prec=1.000, rec=0.743, Spec=nan, auroc_train=1.000, auroc=nan


C:\Users\nshawen\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:48: RuntimeWarning: invalid value encountered in long_scalars



Subj/Visit 3, prec=0.290, rec=0.817, Spec=0.425, auroc_train=1.000, auroc=0.654

Subj/Visit 4, prec=0.650, rec=0.512, Spec=0.430, auroc_train=1.000, auroc=0.446

Subj/Visit 5, prec=0.931, rec=0.827, Spec=0.662, auroc_train=1.000, auroc=0.762

Subj/Visit 6, prec=0.938, rec=0.805, Spec=0.723, auroc_train=1.000, auroc=0.803

Subj/Visit 7, prec=0.640, rec=0.864, Spec=0.351, auroc_train=1.000, auroc=0.595

Subj/Visit 8, prec=0.678, rec=0.441, Spec=0.621, auroc_train=1.000, auroc=0.537

Subj/Visit 9, prec=0.724, rec=0.806, Spec=0.535, auroc_train=1.000, auroc=0.704

Subj/Visit 10, prec=0.709, rec=0.635, Spec=0.628, auroc_train=1.000, auroc=0.665

Subj/Visit 11, prec=0.667, rec=0.986, Spec=0.352, auroc_train=1.000, auroc=0.722

Subj/Visit 12, prec=0.839, rec=0.822, Spec=0.443, auroc_train=1.000, auroc=0.662

Subj/Visit 13, prec=0.072, rec=0.627, Spec=0.386, auroc_train=1.000, auroc=0.471

Subj/Visit 14, prec=0.751, rec=0.729, Spec=0.646, auroc_train=1.000, auroc=0.759

Subj/Visit 15, prec=0.

(           model        f1     auprc     auroc
 0  Random Forest  0.634079  0.648078  0.648485,
 {'Random Forest': {'f1': [0.616279069767442,
    0.48375451263537905,
    0.8525402726146222,
    0.4278846153846154,
    0.5725490196078431,
    0.8756410256410256,
    0.8667642752562225,
    0.7350993377483444,
    0.5342312008978676,
    0.7628657921291625,
    0.6702605570530099,
    0.7954545454545454,
    0.8305274971941637,
    0.1295971978984238,
    0.7397849462365592,
    0.5727272727272726,
    0.15559157212317665,
    0.5957446808510638,
    0.7761485826001955,
    0.6978243352135375,
    0.6754675467546755,
    0.5830065359477125],
   'auprc': [0.5778966478959576,
    0.4574012131006187,
    1.0,
    0.32239691965509715,
    0.6590022115816373,
    0.8974430728078163,
    0.9277419157938327,
    0.6011128767217081,
    0.6852978898134668,
    0.7326462204385609,
    0.7462995132269384,
    0.7416692580464529,
    0.8417970731627411,
    0.06180708140935182,
    0.809971681379

In [18]:
Xtrain_t.Subject.unique()

array([1003, 1004, 1005, 1007, 1009, 1016, 1018, 1019, 1023, 1024, 1029,
       1030, 1032, 1038, 1039, 1043, 1044, 1046, 1048, 1049, 1050, 1051],
      dtype=int64)

## Predict Tremor Scores For Each Task

In [36]:
clf = RandomForestRegressor(n_estimators=100, max_features = 0.5, random_state=2)

groups = Xtrain_t.Subject.values
subj = LeaveOneGroupOut()
X = Xtrain_t.iloc[:,5:].values
indp_t = Xtrain_t['Tremor']
indp_t = indp_t.values
y = indp_t.astype(int)
scores = np.array([])

#LOSO CV for current classifier
for train_index, test_index in subj.split(X, y, groups):
    print(np.unique(Xtrain_t.Subject.values[test_index]))
    Xtr, Xte = X[train_index], X[test_index]
    ytr, yte = y[train_index], y[test_index]
    if len(np.unique(ytr))<2: #skip if no positive or negative examples are available for training
        print('only 1 class available in train data - skipping')
        continue
        
    clf.fit(Xtr,ytr)
    ypred = clf.predict(Xte)
    scores = np.concatenate((scores,ypred))

D = Xtrain_t.copy()
D['Scores'] = pd.Series(scores,index=D.index)
D = D.loc[:,['Subject', 'Visit', 'Task', 'Tremor', 'Scores']]
D.to_pickle('TremorPredictedScores.pkl')

[1003]
[1004]
[1005]
[1007]
[1009]
[1016]
[1018]
[1019]
[1023]
[1024]
[1029]
[1030]
[1032]
[1038]
[1039]
[1043]
[1044]
[1046]
[1048]
[1049]
[1050]
[1051]


## Predict Bradykinesia Scores For Each Task

In [37]:
clf = RandomForestRegressor(n_estimators=100, max_features = 0.5, random_state=2)

groups = Xtrain_b.Subject.values
subj = LeaveOneGroupOut()
X = Xtrain_b.iloc[:,5:].values
indp_b = Xtrain_b['Bradykinesia']
indp_b = indp_b.values
y = indp_b.astype(int)
scores = np.array([])

#LOSO CV for current classifier
for train_index, test_index in subj.split(X, y, groups):
    print(np.unique(Xtrain_b.Subject.values[test_index]))
    Xtr, Xte = X[train_index], X[test_index]
    ytr, yte = y[train_index], y[test_index]
    if len(np.unique(ytr))<2: #skip if no positive or negative examples are available for training
        print('only 1 class available in train data - skipping')
        continue
        
    clf.fit(Xtr,ytr)
    ypred = clf.predict(Xte)
    scores = np.concatenate((scores,ypred))

D = Xtrain_b.copy()
D['Scores'] = pd.Series(scores,index=D.index)
D = D.loc[:,['Subject', 'Visit', 'Task', 'Bradykinesia', 'Scores']]
D.to_pickle('BradykinesiaPredictedScores.pkl')

[1003]
[1004]
[1005]
[1007]
[1009]
[1016]
[1018]
[1019]
[1023]
[1024]
[1029]
[1030]
[1032]
[1038]
[1039]
[1043]
[1044]
[1046]
[1048]
[1049]
[1050]
[1051]
